#Language Learning Assistant (English - French)


##Training

###Setup
Library Installation

In [ ]:
!pip install transformers datasets peft

In [ ]:
!pip install --upgrade transformers peft

Load pretrained T5 model and T5 tokenizer

T5-small model has 60M parameters.

T5-Base: 220M
T5-Large: 770M
T5-3B: 3 billion (3B)
T5-11B: 11 billion (11B)

In [56]:
from transformers import T5ForConditionalGeneration, T5Tokenizer, TrainingArguments, Trainer
from datasets import Dataset
from peft import LoraConfig, get_peft_model, TaskType

# Load pre-trained T5 model and tokenizer.
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)


Sample data for training and validation.

In [57]:

data = {
    "train": [
        {"input_text": "translate English to French: How are you?", "target_text": "Comment ça va?"},
        {"input_text": "translate English to French: I love programming.", "target_text": "J'aime programmer."},
        {"input_text": "translate English to French: This is a beautiful day.", "target_text": "C'est une belle journée."}
    ],
    "validation": [
        {"input_text": "translate English to French: What is your name?", "target_text": "Comment vous appelez-vous?"},
        {"input_text": "translate English to French: I am learning French.", "target_text": "J'apprends le français."}
    ]
}

# Convert to Hugging Face Datasets.
train_dataset = Dataset.from_dict({
    "input_text": [ex["input_text"] for ex in data["train"]],
    "target_text": [ex["target_text"] for ex in data["train"]]
})
val_dataset = Dataset.from_dict({
    "input_text": [ex["input_text"] for ex in data["validation"]],
    "target_text": [ex["target_text"] for ex in data["validation"]]
})


###Implement Low Rank Adaptation
Try out with r=8, Sequence to sequence language model

In [58]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    inference_mode=False,
    r=16,
    lora_alpha=32,
    lora_dropout=0.1
)

# Wrap the model with LoRA.
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()  # Prints the number of trainable parameters.


trainable params: 589,824 || all params: 61,096,448 || trainable%: 0.9654


###Training parameters

Use FP16 for half precision floating point with 16-bit floating point number for faster computation with GPU and enabling mixed precision training

Memory needed: Each parameter in FP16 takes 2 bytes (16 bits)

Total memory needed = Number of parameters × Size per parameter

60
𝑀
×
2
 bytes
=
120
 MB
60M×2 bytes=120 MB

In [59]:
training_args = TrainingArguments(
    output_dir="./t5-lora-language-learning",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_steps=10,
    learning_rate=5e-4,
    weight_decay=0.01,
    fp16=True,  # Enable mixed precision if your GPU supports it.
    push_to_hub=False,
    remove_unused_columns=False,
)




/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


###Preprocess the tokens
Preprocess so that the input and target tokens always have same length


In [60]:
def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input_text"],
        max_length=128,
        truncation=True,
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples["target_text"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


Remove input text and target text columns for training

In [61]:
train_dataset = train_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["input_text", "target_text"]
)
val_dataset = val_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=["input_text", "target_text"]
)

# Ensure only the required columns are returned.
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
val_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/3 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3961: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [62]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, padding=True)


Training setup

In [63]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,  # This will pad your inputs to a uniform length.
)


In [64]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,18.144630
2,No log,17.960712
3,No log,17.646824


TrainOutput(global_step=6, training_loss=17.04507064819336, metrics={'train_runtime': 2.8425, 'train_samples_per_second': 3.166, 'train_steps_per_second': 2.111, 'total_flos': 308595916800.0, 'train_loss': 17.04507064819336, 'epoch': 3.0})

###Save the trained model and the tokenizer

In [65]:
model.save_pretrained("./t5-lora-language-learning")
tokenizer.save_pretrained("./t5-lora-language-learning")


('./t5-lora-language-learning/tokenizer_config.json',
 './t5-lora-language-learning/special_tokens_map.json',
 './t5-lora-language-learning/spiece.model',
 './t5-lora-language-learning/added_tokens.json')

##Evaluate the trained model

###Small dataset evaluation

In [28]:
!pip install rouge-score

In [66]:
import math
import torch
import nltk
nltk.download('punkt')
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sample test dataset: list of dictionaries with English and reference French translations.

In [30]:
test_data = [
    {"english": "Good morning", "french": "Bonjour"},
    {"english": "How are you?", "french": "Ça va ?"},
    {"english": "I love you", "french": "Je t'aime"},
    {"english": "Thank you", "french": "Merci"},
    {"english": "Good night", "french": "Bonne nuit"}
]

Generate predictions for the test set

In [31]:
predictions = []  # list of tokenized predicted sentences (lists of words)
references = []   # list of lists (each inner list containing one tokenized reference sentence)
for sample in test_data:
    # Create prompt for translation.
    prompt = f"translate English to French: {sample['english']}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model.generate(**inputs, max_length=50)
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Tokenize predictions and references.
    predictions.append(pred_text.split())
    references.append([sample['french'].split()])  # corpus_bleu expects a list of references for each prediction

Compute BLEU score using NLTK

In [37]:
smoothie = SmoothingFunction().method2
bleu = corpus_bleu(references, predictions, smoothing_function=smoothie)
print("BLEU score:", bleu)

BLEU score: 0.22416933501922293


Compute ROUGE scores using the rouge_score library

In [24]:
from rouge_score import rouge_scorer, scoring

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
aggregator = scoring.BootstrapAggregator()

for sample in test_data:
    prompt = f"translate English to French: {sample['english']}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model.generate(**inputs, max_length=50)
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    scores = scorer.score(sample['french'], pred_text)
    aggregator.add_scores(scores)
rouge_scores = aggregator.aggregate()
print("ROUGE scores:")
for key, val in rouge_scores.items():
    print(f"  {key}: {val.mid.fmeasure:.4f}")

ROUGE scores:
  rouge1: 0.5333
  rouge2: 0.2000
  rougeL: 0.5333


Compute Perplexity over the test set

In [25]:
losses = []
for sample in test_data:
    prompt = f"translate English to French: {sample['english']}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    # Tokenize reference French sentence as labels.
    labels = tokenizer(sample['french'], return_tensors="pt", padding=True, truncation=True, max_length=128).input_ids
    # Replace pad tokens with -100 so they are ignored in loss computation.
    labels[labels == tokenizer.pad_token_id] = -100
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(**inputs, labels=labels)
    losses.append(outputs.loss.item())

avg_loss = sum(losses) / len(losses)
perplexity = math.exp(avg_loss)
print("Perplexity:", perplexity)


Perplexity: 3.151670484517875


###Evaluate with Larger dataset

Load the WMT14 dataset using the "fr-en" configuration.
Then swap the roles to evaluate english-to-french translation.

In [39]:
import math
import torch
import nltk
from datasets import load_dataset
nltk.download('punkt')
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

dataset = load_dataset("opus100", "en-fr", split="test")

test_data = dataset.select(range(50))

# Prepare lists to accumulate predictions and references for BLEU.
predictions = []  # Each prediction is tokenized (list of words)
references = []   # Each reference is a list of one tokenized reference sentence

# Loop over the test dataset.
for sample in test_data:
    # In the "fr-en" config, 'translation' is a dict with keys "fr" and "en".
    # For english-to-french translation, we treat the "en" text as the input
    # and the "fr" text as the reference translation.
    english_sentence = sample["translation"]["en"]
    french_reference = sample["translation"]["fr"]

    prompt = f"translate English to French: {english_sentence}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model.generate(**inputs, max_length=50)
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    predictions.append(pred_text.split())
    references.append([french_reference.split()])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/327k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/142M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/334k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [67]:
test_data = dataset.select(range(100))

# Prepare lists to accumulate predictions and references for BLEU.
predictions = []  # Each prediction is tokenized (list of words)
references = []   # Each reference is a list of one tokenized reference sentence

# Loop over the test dataset.
for sample in test_data:
    # In the "fr-en" config, 'translation' is a dict with keys "fr" and "en".
    # For english-to-french translation, we treat the "en" text as the input
    # and the "fr" text as the reference translation.
    english_sentence = sample["translation"]["en"]
    french_reference = sample["translation"]["fr"]

    prompt = f"translate English to French: {english_sentence}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model.generate(**inputs, max_length=50)
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    predictions.append(pred_text.split())
    references.append([french_reference.split()])

Compute BLEU and ROGUE score

In [70]:
smoothie = SmoothingFunction().method2
bleu = corpus_bleu(references, predictions, smoothing_function=smoothie)
print("BLEU score:", bleu)

from rouge_score import rouge_scorer, scoring

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
aggregator = scoring.BootstrapAggregator()

for sample in test_data:
    english_sentence = sample["translation"]["en"]
    french_reference = sample["translation"]["fr"]

    prompt = f"translate English to French: {english_sentence}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
    outputs = model.generate(**inputs, max_length=50)
    pred_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    scores = scorer.score(french_reference, pred_text)
    aggregator.add_scores(scores)

rouge_scores = aggregator.aggregate()
print("ROUGE scores:")
for key, val in rouge_scores.items():
    print(f"  {key}: {val.mid.fmeasure:.4f}")



BLEU score: 0.2642816955517258
ROUGE scores:
  rouge1: 0.5135
  rouge2: 0.3407
  rougeL: 0.4919


##Setup for CUDA

In [12]:
import torch

# Determine device and move the model to the device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Prepare the input and move it to the same device.
input_text = "translate English to French: I am learning to code."
inputs = tokenizer(input_text, return_tensors="pt").to(device)

# Generate output on the same device.
outputs = model.generate(**inputs, max_length=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


Je apprends à coder.


##Model deployment

###Install local tunnel

In [13]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 3s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

###Local Tunnel setup

In [14]:
!npm config set prefix ~/.npm-global
!export PATH=~/.npm-global/bin:$PATH
!npm install -g localtunnel


⠙⠹⠸⠼
added 22 packages in 735ms
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

###Translator

In [ ]:
import threading
from flask import Flask, request, render_template_string
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize Flask app.
app = Flask(__name__)

# Set the directory where your fine-tuned model is stored.
model_dir = "./t5-lora-language-learning"  # Adjust this path if needed.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the tokenizer and model.
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
model.to(device)

# Define a simple HTML template for the translator interface.
html_template = """
<!doctype html>
<html>
<head>
  <title>English to French Translator</title>
</head>
<body>
  <h1>English to French Translator</h1>
  <form method="POST">
    <textarea name="input_text" rows="4" cols="50" placeholder="Enter text in English..."></textarea><br>
    <input type="submit" value="Translate">
  </form>
  {% if output_text %}
    <h2>Translated Text:</h2>
    <p>{{ output_text }}</p>
  {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def translate():
    output_text = None
    if request.method == 'POST':
        input_text = request.form['input_text']
        # Prepend the task-specific prefix required by T5.
        prompt = f"translate English to French: {input_text}"
        # Tokenize the input and move tensors to the appropriate device.
        inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
        # Generate the translation.
        outputs = model.generate(**inputs, max_length=50)
        output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return render_template_string(html_template, output_text=output_text)

def run_app():
    # Run on all available IP addresses, port 5000.
    app.run(host="0.0.0.0", port=5000)

# Start the Flask app in a background thread.
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'


###Gamification

In [15]:
import threading
from flask import Flask, session, request, render_template_string, url_for
import random
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

app = Flask(__name__)
app.secret_key = "some"  # Replace with a secure key

# Load your fine-tuned model and tokenizer.
model_dir = "./t5-lora-language-learning"  # Adjust the path as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)
model.to(device)

# List of English sentences for the game.
english_sentences = [
    "Good morning",
    "How are you?",
    "I love you",
    "Thank you",
    "Good night"
]

# A bank of common French phrases to serve as distractors.
distractor_bank = [
    "Bonsoir",
    "Merci beaucoup",
    "Bonne nuit",
    "Au revoir",
    "Comment ça va?",
    "Je ne sais pas",
    "S'il vous plaît",
    "Pardon"
]

def generate_translation(english_sentence):
    """Generate French translation using the fine-tuned model."""
    prompt = f"translate English to French: {english_sentence}"
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)
    outputs = model.generate(**inputs, max_length=50)
    translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translation

@app.route("/", methods=["GET", "POST"])
def index():
    # Initialize session variables if not already set.
    if "score" not in session or "current_question" not in session:
        session["score"] = 0
        session["current_question"] = 0

    current = session["current_question"]

    # If all questions have been answered, display final score.
    if current >= len(english_sentences):
        final_score = session["score"]
        total = len(english_sentences)
        session.clear()  # Clear session to allow a new game.
        return render_template_string("""
            <h1>Game Over</h1>
            <p>Your final score is: {{ score }} out of {{ total }}</p>
            <a href="{{ url_for('index') }}">Play Again</a>
        """, score=final_score, total=total)

    # Process answer submission.
    if request.method == "POST":
        selected = request.form.get("option")
        correct_answer = session.get("correct_answer", "")
        if selected == correct_answer:
            session["score"] += 1
            feedback = "Correct!"
        else:
            feedback = f"Incorrect. The correct answer was: {correct_answer}"
        session["current_question"] = current + 1
        return render_template_string("""
            <h1>{{ feedback }}</h1>
            <p>Current Score: {{ score }}</p>
            <a href="{{ url_for('index') }}">Next Question</a>
        """, feedback=feedback, score=session["score"])

    # For GET requests, generate a new question.
    english_sentence = english_sentences[current]
    correct_translation = generate_translation(english_sentence)
    session["correct_answer"] = correct_translation

    # Prepare distractors by sampling two options (ensuring they're different from the correct answer).
    available_distractors = [d for d in distractor_bank if d.lower() != correct_translation.lower()]
    if len(available_distractors) < 2:
        available_distractors = distractor_bank
    distractors = random.sample(available_distractors, 2)
    options = distractors + [correct_translation]
    random.shuffle(options)

    return render_template_string("""
        <h1>Translate to French</h1>
        <p><strong>English:</strong> {{ english_sentence }}</p>
        <form method="POST">
            {% for opt in options %}
                <input type="radio" id="{{ opt }}" name="option" value="{{ opt }}" required>
                <label for="{{ opt }}">{{ opt }}</label><br>
            {% endfor %}
            <br>
            <input type="submit" value="Submit">
        </form>
        <p>Current Score: {{ score }}</p>
    """, english_sentence=english_sentence, options=options, score=session["score"])

def run_app():
    # Disable debug and reloader to avoid recursion errors in Colab.
    app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)

# Start the Flask app in a background thread.
thread = threading.Thread(target=run_app)
thread.start()


 * Serving Flask app '__main__'


###Get the local tunnel password

In [17]:
!curl https://loca.lt/mytunnelpassword

34.87.44.15

###Deploy
Run local tunnel

In [18]:
!/root/.npm-global/bin/lt --port 5000


your url is: https://cruel-rooms-cough.loca.lt


INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:06] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:15] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:17] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:30] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:32] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:41] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:55] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Mar/2025 18:39:57] "GET / HTTP/1.1" 200 -


^C
